In [1]:
indir = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_unique_constraints_None/clonal_shifts/variants_prefilterMerge_impute/donors/donor2/clones/knn_kparam_30"
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_unique_constraints_None/repr_clones/donor2/cloneMethod_variants_prefilterMerge_impute_knn_resolution_30/clonalShift_method_clones/clones_ranked"
cells_meta_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_unique_constraints_None/repr_clones/donor2/cloneMethod_variants_prefilterMerge_impute_knn_resolution_30/clonalShift_method_clones/cells_meta.tsv"
is_mt = False 
condition = "None"
clone_id = "cloneID" # "den_clust"
p_thresh = 0.1 


cells_meta_f = "" #"/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_pct/repr_clones/donor2/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/cells_meta.tsv"


In [2]:
%load_ext autoreload
%autoreload 2
from glob import glob
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

import seaborn as sns
import matplotlib.pyplot as plt

from src import clonal_shifts as cs
from icecream import ic

from mplh import cluster_help as ch

from scipy.stats import rankdata, gmean


fig_utils from mplh


## Load ncells and save to output. Sort by ncells

In [4]:
noInput_ncells = pd.read_csv(join(indir, "noInput_ncells.csv"),  index_col=0)
input_ncells = pd.read_csv(join(indir, "input_ncells.csv"), index_col=0)

def get_clone_ncells(df):
    if df.shape[0] == 0:
        df = pd.DataFrame(columns=["count"])
        df.index.name = clone_id
        return df
    return df.groupby(clone_id).sum()[["count"]].sort_values(by="count")[::-1]


input_ncells = get_clone_ncells(input_ncells)#.groupby(clone_id).sum()[["count"]].sort_values(by="count")[::-1]
noInput_ncells = get_clone_ncells(noInput_ncells) #.groupby(clone_id).sum()[["count"]].sort_values(by="count")[::-1]
ncells_df = pd.merge(input_ncells,noInput_ncells,left_index=True, right_index=True, suffixes=["_input", "_cultured"],
                    how="outer").fillna(0)
ncells_df = ncells_df.loc[ncells_df.sum(axis=1).sort_values()[::-1].index]
ncells_df

,count_input,count_cultured
cloneID,,


## Norm ncells for each and get ratio

In [5]:
norm_ncells_df = ncells_df/ncells_df.sum(axis=0)
norm_ncells_df

norm_ncells_df["fold"] = norm_ncells_df["count_cultured"]/norm_ncells_df["count_input"].fillna(0)
norm_ncells_df

,count_input,count_cultured,fold
cloneID,,,


## Load shuffle hypergeometric results for input and cultured
## Convert into long df with cloneID ('index'), clusterID ('variable'), pval, method (hypergeo or one of four shuffle methods), and condition ('input' or 'cultured')

In [6]:
noInput_hyper = pd.read_csv(join(indir, "noInput_hypergeo_pvals.csv"),  index_col=0)
noInput_shuffle_df = pd.read_csv(join(indir, "noInput_shuffle_results_pvals.csv"), index_col=0)
noInput_shuffle_df = noInput_shuffle_df.rename({"value":"pval"}, axis=1)

input_hyper = pd.read_csv(join(indir, "input_hypergeo_pvals.csv"), index_col=0)
input_shuffle_df = pd.read_csv(join(indir, "input_shuffle_results_pvals.csv"), index_col=0)

input_shuffle_df = input_shuffle_df.rename({"value":"pval"}, axis=1)
input_shuffle_df

input_hyper_df = input_hyper.reset_index().melt(id_vars="index", value_name="pval")
input_hyper_df["method"] = "hypergeo"
noInput_hyper_df = noInput_hyper.reset_index().melt(id_vars="index", value_name="pval")
noInput_hyper_df["method"] = "hypergeo"


noInput_hyper_df["condition"] = "cultured"
noInput_shuffle_df["condition"] = "cultured"
input_hyper_df["condition"] = "input"
input_shuffle_df["condition"] = "input"


In [7]:
input_df = pd.concat((input_hyper_df, input_shuffle_df), axis=0, ignore_index=True)
input_df["variable"] = input_df["variable"].astype(object).astype(str)

noInput_df = pd.concat((noInput_hyper_df, noInput_shuffle_df), axis=0, ignore_index=True)
noInput_df["variable"] = noInput_df["variable"].astype(object).astype(str)


df = pd.concat((input_df, noInput_df), axis=0).reset_index(drop=True)
#df = pd.merge(input_df, noInput_df, on=["index", "variable", "method"], suffixes=["_input", "_cultured"])
df

,index,variable,pval,method,condition


## Create 'sig score' that merges the hypergeo and the shuffle results. Score 0-4, with 4 the highest

In [14]:
p_df_group, p_df_out = cs.hypergeo_score(df, p_thresh )
p_df_group

(0, 6)
False
(0, 6)
Empty DataFrame
Columns: [index, variable, pval, method, condition, is_sig]
Index: []
None
p_df_out
Empty DataFrame
Columns: [index, variable, pval, method, condition, is_sig]
Index: []
p_df_out after
Empty DataFrame
Columns: [index, variable, pval, method, condition, is_sig]
Index: []


ValueError: Wrong number of items passed 6, placement implies 1

In [ ]:
p_df_out

In [ ]:
p_df_out.sum()

In [ ]:
sns.countplot(data=p_df_group.reset_index(), x=0, hue="condition")
plt.savefig(join(outdir, "hypergeo_sig_score_distribution.pdf"))


## Save heatmap for initial hypergeo 

In [ ]:
clone_meta = pd.DataFrame(index=p_df_out.columns, columns = ["condition"])
clone_meta["condition"] = ["input" if "input" in x else "cultured" for x in clone_meta.index]
clone_meta

## Input and culture in same heatmap- show clones that were significant in i'th Input cluster label and j'th culture cluster label

In [ ]:
a_input = df.loc[(df["method"] == "hypergeo") & (df["condition"] == "input")].pivot(index="index", columns="variable", values="pval")
b_cultured = df.loc[(df["method"] == "hypergeo") & (df["condition"] == "cultured")].pivot(index="index", columns="variable", values="pval")
merged_df, merged_count_df = cs.merge_hypergeom(a_input, b_cultured, 
                                             "Input", "Cultured", p_thresh=p_thresh,
                                             f_save=join(outdir, f"culture_input_clone_lineages"))


In [ ]:
a_sig = p_df_out[clone_meta[clone_meta["condition"]=="input"].index]
b_sig = p_df_out[clone_meta[clone_meta["condition"]=="cultured"].index]
a_sig.columns = [x.replace("_cultured","").replace("_input","") for x in a_sig.columns]
b_sig.columns = [x.replace("_cultured","").replace("_input","") for x in b_sig.columns]

In [ ]:
merged_df, merged_count_df = cs.merge_hypergeom(1/(1+a_sig), 1/(1+b_sig), 
                                               "Input", "Cultured", p_thresh=0.25,
                                                title="Clones sig in a cluster in both the input and culture (score=4/4)",
                                                f_save=join(outdir,f"sig_score4_culture_input_clone_lineages"))


merged_df, merged_count_df = cs.merge_hypergeom(1/(1+a_sig), 1/(1+b_sig), 
                                                "Input", "Cultured", p_thresh=0.33,
                                                title="Clones sig in a cluster in both the input and culture (score=3/4)",
                                                f_save=join(outdir,f"sig_score3_culture_input_clone_lineages"))


merged_df, merged_count_df = cs.merge_hypergeom(1/(1+a_sig), 1/(1+b_sig), 
                                             "Input", "Cultured", p_thresh=0.5,
                                             title="Clones sig in a cluster in both the input and culture (score=2/4)",
                                             f_save=join(outdir,f"sig_score2_culture_input_clone_lineages"))


merged_df, merged_count_df = cs.merge_hypergeom(1/(1+a_sig), 1/(1+b_sig), 
                                             "Input", "Cultured", p_thresh=1,
                                             title="Clones sig in a cluster in both the input and culture (score=1/4)",
                                             f_save=join(outdir,f"sig_score1_culture_input_clone_lineages"))


## Get the minimum of each clone shift. This will be used for rankigng later

In [ ]:
p_df_sep = p_df_group.reset_index().groupby(["index", "condition"]).sum().reset_index()
p_df_sep

In [ ]:
def sum_hyper_score(ser):
    out = {}
    tot = 0
    for ind, val in p_df_sep[p_df_sep["index"]==ser.name].iterrows():
        out[f'hyper_score_{val["condition"]}'] = val[0]
        tot += val[0]
    out["hyper_score"] = tot
    return out



min_df = df.groupby(["index", "method", "condition"]).min()["pval"].reset_index().sort_values("pval")
min_df

min_df_wide = min_df.pivot(index="index", values="pval", columns=["method", "condition"])
min_df_wide.columns = ['_'.join(col).strip() for col in min_df_wide.columns.values] # flatten to 1D columns
min_df_wide
ncells_df["ncells"] = ncells_df.sum(axis=1)
min_df_wide = pd.concat((ncells_df, norm_ncells_df[["fold"]], min_df_wide), axis=1)
# min_df_wide = pd.merge(min_df_wide, norm_ncells_df, left_index=True, right_index=True, 
#                       suffixes=("", "_norm"))


if is_mt:
    print("removing donor specific variants")
    print(min_df_wide.shape)
    clone_order_keep = []
    cells_meta = pd.read_csv(cells_meta_f, sep="\t", index_col=0)
    kept_clones = []
    kept_clones_cells = []
    
    for x in cells_meta.columns:
        if "cloneID_" in x:
            kept_clones_cells.append(x)
            kept_clones.append(x.replace("cloneID_", ""))
    
    min_df_wide = min_df_wide.loc[min_df_wide.index.isin(kept_clones)]
    print("after removal")
    print(min_df_wide.shape)

min_df_wide = min_df_wide.fillna(0)
min_df_wide = min_df_wide.sort_values("ncells", ascending=False)

min_df_wide = pd.concat((min_df_wide.apply(sum_hyper_score, result_type='expand', axis=1), min_df_wide ), axis=1)
min_df_wide

frst_cols = ["ncells", "hyper_score", "fold"]
min_df_wide = min_df_wide[frst_cols + [i for i in min_df_wide.columns.values if i not in frst_cols]]
min_df_wide

# *Get the representative clones:
a. top 3 by size  
b. bottom 2 by size (min 10)  
c. top 2 by clonal shift score  
d. top 2 and bottom 2 by fold  

Save how they were chosen. 
Put in this order (but if there's duplicates, drop those)




In [ ]:
def add_to_ord(k, out_ord):
    if k not in out_ord:
        out_ord.append(k)
    return out_ord

def rep_clones(df):
    out_cl = {}
    out_ord = []
    
    # ncells
    
    ## top ncells
    ncells = df['ncells'].sort_values()[::-1]
    
    print('ncells')
    print(ncells.shape)
    out_cl[("ncells", 1) ] = ncells.index[0]
    out_ord.append(ncells.index[0])
    
    if ncells.shape[0] > 1:
        out_cl[("ncells", 2)] = ncells.index[1]
        add_to_ord(ncells.index[1], out_ord)
    
    ## bottom ncells
    ncells = ncells[ncells>10]
    #if ncells
    if len(ncells.shape) > 2: #more than just top 2
        out_cl[("smallest_ncells", 1)] = ncells.index[-1]
        add_to_ord(ncells.index[-1], out_ord)
        if len(ncells.shape) > 3:
            out_cl[("smallest_ncells", 2)] = ncells.index[-2]
            add_to_ord(ncells.index[-2], out_ord)
    
    # hyper score
    hyper_score = df['hyper_score'].sort_values()[::-1]
    hyper_score = hyper_score[hyper_score>0]
    for i in range(min(2, len(hyper_score))):
        out_cl[("hyper", i+1)] = hyper_score.index[i]
        add_to_ord(hyper_score.index[i], out_ord)
        
    # fold in culture
    fold = df['fold'].sort_values()[::-1]
    out_cl[("culture_fold", 1) ] = fold.index[0]
    add_to_ord(fold.index[0], out_ord)
    if fold.shape[0] > 1:    
        out_cl[("culture_fold", 2)] = fold.index[1]
        add_to_ord(fold.index[1], out_ord)
    
    return out_cl, out_ord

clone_reps, clone_rep_order = rep_clones(min_df_wide)
print(clone_rep_order)
clone_reps


## Save

In [ ]:
with open(join(outdir, "representative_cloneID.txt"), 'w') as f:
    f.write('\n'.join(clone_rep_order))

pd.Series(clone_reps).reset_index().to_csv(join(outdir, "representative_cloneID.csv"), header=None, 
                                           index=False)

